<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/Unidad_3/5_Memoizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np

# Memoización

En esta libreta veremos algunos ejemplos de uso de memoización.

## Sucesión de Fibonacci

Empecemos con un ejemplo que ya habíamos visto. Una función recursiva sencilla para calcular el n-ésimo número de Fibonacci es:

In [1]:
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

Esta función realiza muchas llamadas redundantes. Por ejemplo, si queremos calcular $F_4$, el árbol de recursión asociado será:

```
            f(4)
        /         \
      f(3)       f(2)
     /    \     /    \
   f(2)  f(1) f(1)  f(0)
  /    \
f(1)  f(0)
```
Podemos ver que hace dos llamadas a `f(2)`; conforme más crece `n`, habrá aun más llamadas redundantes.

Para solucionar esto, podemos inicializar una lista vacía `L` de tamaño `n`. El elemento $L_i$ tendrá el resultado de `f(i)`. Al inicio, todos los elementos de `L` serán `None`, con la excepción de `L[0]=0`, y `L[1]=1`. En la primera llamada de `f(i)`, llenaremos el correspondiente `L[i]`, y en todas las subsecuentes simplemente consultaremos el valor guardado.

In [4]:
def fib_memo(n, L=None):
    if L is None:
        L = [None] * (n+1)
        L[0] = 0
        L[1] = 1
    if L[n] is not None:
        return L[n]
    else:
        L[n] = fib_memo(n-1, L) + fib_memo(n-2, L)
        return L[n]

Podemos comparar los tiempos de ejución para números grandes:

In [5]:
n = 30
%timeit fib(n)
%timeit fib_memo(n)

360 ms ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
11.4 µs ± 553 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Mochilas

Supongamos que tenemos una lista de artículos, cada uno con pesos $W=\{w_i\}$ y valores $V=\{v_i\}$. Nosotros podemos cargar como máximo $w$ kilos. ¿Cuál es la combinación de artículos que maximiza nuestras ganancias?

A este y otros problemas similares se les conoce como *problemas de mochila*. Específicamente, esta es la variante $0-1$, ya que las únicas opciones son llevar un artículo ($1$) o no hacerlo ($1$). En otras versiones, podemos llevar partes de artículos, o tenemos una mochila infinita.

Podemos resolver este problema utilizando *backtracking*:

* Empezamos con una mochila vacía y vamos pasando por cada uno de los artículos haciendo dos llamadas recursivas: una en la que sí lo tomamos, y otra en la que no. 
* Regresamos el máximo de las ganancias de las dos llamadas.
* Los casos base son cuando ya no podemos cargar nada más, o acabamos todos los artículos.

In [30]:
def knapsack(weights, values, capacity, idx=0):
    if capacity <= 0 or idx >= len(values):
        return 0
    
    if weights[idx] <= capacity:
        new_capacity = capacity - weights[idx]        
        p1 = values[idx] + knapsack(weights, values, new_capacity, idx+1) 
    else:        
        p1 = 0        
        
    p2 = knapsack(weights, values, capacity, idx+1)
    
    return max(p1, p2)

In [31]:
weights = [1, 2, 3, 5]
values = [1, 6, 10, 16]
capacity = 7
knapsack(weights, values, capacity)

22

Podemos mejorar esto utilizando memoización. En las llamadas recursivas, tenemos dos valores cambiantes: la capacidad y el índice. Entonces, creamos un arreglo `memo` de tamaño $(\text{capacidad}+1)\times (\text{largo de las listas})$ (ya que la capacidad va de $0$ hasta la inicial y el índice de $0$ al tamaño de las listas). La entrada `memo[i,j]` tendrá la ganancia máxima para la capacidad `i` y el índice `j`:

In [97]:
def knapsack_memo(weights, values, capacity, idx=0, memo=None):
    if memo is None:
        memo = [[None for _ in range(len(weights))] for _ in range(capacity+1)]
    
    if capacity <= 0 or idx >= len(values):
        return 0
    
    if memo[capacity][idx] is not None:
        return memo[capacity][idx]
    else:
        p1 = 0
        if weights[idx] <= capacity:
            new_capacity = capacity - weights[idx]
            p1 = values[idx] + knapsack_memo(weights, values, new_capacity, idx+1, memo)
        p2 = knapsack_memo(weights, values, capacity, idx+1, memo)
        
        memo[capacity][idx] = max(p1, p2)
    return memo[capacity][idx]

In [98]:
weights = [1, 2, 3, 5]
values = [1, 6, 10, 16]
capacity = 7
knapsack_memo(weights, values, capacity)

22

Notemos que el resultado que queremos es el elemento `memo[capacidad, largo de listas]`, ya que dicho elemento representa la ganancia máxima posible dada la capacidad `capacidad`, y estando parados en el índice `largo de listas` (i.e., cuando recorrimos las listas por completo).

Entonces, podemos quitar nuestras llamadas recursivas y simplemente enfocarnos en llenar el arreglo completo, después de lo cual extraemos el elemento que buscamos. Para hacer esto, primero vemos que para cada entrada `memo[c, i]`, tenemos dos opciones:

1. Excluir el artículo en el índice `i`, en cuyo caso la ganancia `memo[c, i]` será igual a la ganancia en `memo[c, i-1]`.
2. Tomar el artículo en el índice `i` (si su peso lo permite). En este caso, la ganancia será igual al precio del artículo `i`, mas la ganancia máxima de los artículos anteriores, dada la nueva capacidad: `valor[i] + memo[c-pesos[i], i-1]`

La entrada `memo[c,i]` será el máximo de las dos cantidades anteriores.

Finalmente, los casos base son: 

* Tenemos una capacidad de cero, en cuyo caso la ganancia máxima posible es cero. Es decir, `memo[0, i] = 0`.
* Tenemos solo un elemento, en cuyo caso la ganancia será igual al precio del elemento (si podemos cargarlo) y 0 en otro caso.
* (Como nota: `memo[0,0] = 0`)

In [87]:
def knapsack_memo_2(weights, values, capacity, memo=None):
    if memo is None:
        memo = [[None for _ in range(len(weights))] for _ in range(capacity+1)]
        
        for c in range(capacity+1):
            memo[c][0] = values[0] if weights[0] <= c else 0

        for i in range(len(weights)):
            memo[0][i] = 0
    for i in range(1, len(weights)):
        for c in range(1, capacity+1):
            p1 = memo[c][i-1]
            p2 = 0
            if weights[i] <= c:
                p2 = values[i] + memo[c-weights[i]][i-1]
            memo[c][i] = max(p1, p2)
    return memo[capacity][len(weights)-1]

In [88]:
knapsack_memo_2(weights, values, capacity)

22

Esta técnica de eliminar llamadas recursivas y simplemente llenar un arreglo se utiliza mucho en memoización, ya que es más eficiente.

# Ejercicios

Dadas dos cadenas de caracteres $X=x_1x_2\ldots x_n$, $Y=y_1y_2\ldots y_m$, la *distancia de edición* es el costo mínimo de *operaciones de edición* que debemos de realizar para convertir la cadena $X$ a $Y$. Las operaciones de edición son:

* Borrar una letra, con costo $\delta$
* Insertar una letra, con costo $\delta$
* Cambiar una letra $u$ a una letra $v$, con costo $\alpha(u, v)$

Ahora, denotemos por $f(i,j)$ como la distancia de edición entre la cadena $X_i=x_1x_2\ldots x_i$ y $Y_j=x_1x_2\ldots x_j$, con $0\leq i\leq n$ y $0\leq j\leq m$. 

Claramente $f(n,m)$ es la distancia de edición entre las palabras originales. Por otro lado, si $i=0$, tenemos que $X_i$ va a ser igual a la cadena vacía, por lo cual tendremos que insertar $j$ caracteres para volverla $Y_j$. Por lo tanto:
$$
f(0,j) = \delta\cdot j
$$

Análogamente, si $j=0$, $Y_j$ será la cadena vacía ($\epsilon$), por lo cual tendremos que borrar todos los caracteres de $X_i$, que es equivalente a $i$ operaciones de borrado. Así:
$$
f(i,0) = \delta\cdot i
$$

Por otro lado, si $i\neq 0, j\neq 0$, consideremos $x_i, y_j$, los últimos caracteres de $X_i$ y $Y_j$, respectivamente. Si queremos que ambos caracteres sean iguales, tenemos que realizar una de las tres operaciones de edición para cambiarlos. Como estamos buscando el costo de edición *mínimo*, la distancia de edición será igual al costo mínimo de estas tres operaciones.

Para entender cómo se obtienen las expresiones recursivas, consideremos un ejemplo con `X=HOLA` y `Y=POZO`. Como mencionamos anteriormente, tenemos tres opciones:

1. Insertar la úlima letra de `Y` al final de `X`. Con esto, las palabras se vuelven:

```
HOLAO
 POZO
```

Ahora, los últimos caracteres son iguales, por lo cual podemos ignorarlos. Como la inserción tiene un costo `d`, la distancia será:

```
f(HOLA, POZO) = f(HOLA, POZ) + d
```

Podemos ver que en este caso, la palabra `X` se mantuvo igual, mientras que la palabra `Y` perdió su última letra. Por lo tanto, en general podemos expresar el costo de esta operación como:

$$
f(i,j) = f(i, j-1) + \delta
$$

2. Borrar la última letra de `X`. Con esto, las palabras son:

```
HOL*
POZO
```

En este caso, fue la palabra `X` la que cambió de largo, por lo cual el costo es:

```
f(HOLA, POZO) = f(HOL, POZO) + d
```

O, en general:

$$
f(i, j) = f(i-1, j) + \delta
$$

Uno puede preguntarse de qué sirve borrar una letra. Para entender por qué, basta con imaginar qué pasa si tenemos palabras de distintos largos; por ejemplo, `X=CARTERA` y `Y=TAZA`. Podemos imaginar que colocamos caracteres vacíos (`*`) al final de `TAZA` para alinearla con `CARTERA`:
```
CARTERA
TAZA***
```
Con esto, podemos considerar a `*` como una letra más, y hacer todas las comparaciones necesarias. 

3. Reemplazar la última letra de `X` por la última letra de `Y`, es decir:

```
HOLO
POZO
```

Recordando que el reemplazo tiene un costo `a(u,v)`, donde `u` es la letra original y `v` la nueva, tenemos que el costo es:

```
f(HOLA, POZO) = f(HOL, POZ) + a(A, O)
```

Ambas palabras perdieron su última letra, por lo tanto:

$$
f(i, j) = 
f(i-1, j-1) + \alpha(x_i, y_j)
$$

Finalmente, la distancia de edición será el mínimo de las tres cantidades anteriores. Añadiendo los casos base que explicamos previamente, llegamos a la expresión recursiva completa:

$$
f(i, j) = 
\begin{cases}
\delta\cdot i & j = 0\\
\delta \cdot j & i = 0\\
\min
\begin{cases}
\alpha(x_i, y_j) + f(i-1, j-1) \\
\delta + f(i-1, j) \\
\delta + f(i, j-1) 
\end{cases} & \text{en otro caso}
\end{cases}
$$

## Ejercicio 1

Utilizando la expresión anterior, escribe en la siguiente celda una función recursiva que calcule la distancia de edición entre dos cadenas. Asume que el costo de reemplazar cualquier par de caracteres es $a$ (i.e. , $\alpha(x, y)=a$, con la excepción de $x=y$, en cuyo caso vale cero). No utilices memoización.

In [243]:
def edit_distance(X, Y, m, n, a=1, d=1):
  
    #Si la primera cadena es vacia, la unica opcion
    #es insertar todos los caracteres de la segunda cadena. 
    if m == 0:
        return n * d
  
    #Si la segunda cadena es vacia, la unica opcion
    #es eliminar todos los caracteres de la primera cadena.
    if n == 0:
        return m * d
  
    # Si los ultimos caractares de las dos cadenas son los mismo.
    if X[m-1] == Y[n-1]:
        return edit_distance(X, Y, m-1, n-1, a, d) + 0 
  
    # Si los ultimos caractares son diferentes, considerar
    # todas las posibilidades y encontrar el minimo.
    return min(edit_distance(X, Y, m, n-1, a, d)+d,
                   edit_distance(X, Y, m-1, n, a, d)+d,
                   edit_distance(X, Y, m-1, n-1, a, d)+a)

Ejecuta la siguiente celda sin cambiar nada, y compara tus resultados:

In [245]:
X, Y = "casa", "raza"
d = edit_distance(X, Y, len(X), len(Y))
print(f"Distancia obtenida: {d}. Esperada: 2")

X, Y = "ola", "hola"
d = edit_distance(X, Y, len(X), len(Y))
print(f"Distancia obtenida: {d}. Esperada: 1")

X, Y = "silla", "escritorio"
d = edit_distance(X, Y, len(X), len(Y), d = 3, a = 4)
print(f"Distancia obtenida: {d}. Esperada: 27")

Distancia obtenida: 2. Esperada: 2
Distancia obtenida: 1. Esperada: 1
Distancia obtenida: 27. Esperada: 27


## Ejercicio 2

Realiza memoización parcial, es decir, solo crea una tabla de memoización, y guarda los resultados, pero no cambies la estructura recursiva (como lo que hicimos en `knapsack_memo`).

In [249]:
def edit_distance_2(X, Y, m, n, a=1, d=1, memo=None):
    
    # Creacion de una tabla para almacenar los resultados de los subproblemas.
    if memo is None:        
        memo = [[None for x in range(n + 1)] for x in range(m + 1)]

    #Si la primera cadena es vacia, la unica opcion
    #es insertar todos los caracteres de la segunda cadena  
    if m == 0:
        return n * d

    #Si la segunda cadena es vacia, la unica opcion
    #es eliminar todos los caracteres de la primera cadena      
    if n == 0:
        return m * d

    # Si los ultimos caractares de las dos cadenas son los mismo. 
    if X[m-1] == Y[n-1]:
        return edit_distance_2(X, Y, m-1, n-1, a, d) + 0

    # Si los ultimos caractares son diferentes, considerar
    # todas las posibilidades y encontrar el minimo.    
    if memo[m][n] is not None:
        return memo[m][n]
    else:
        memo[m][n] = min(edit_distance_2(X, Y, m, n-1, a, d)+d,    # Insert
                   edit_distance_2(X, Y, m-1, n, a, d)+d,    # Remove
                   edit_distance_2(X, Y, m-1, n-1, a, d)+a)
        return memo[m][n]

Verifica tus resultados:

In [248]:
X, Y = "casa", "raza"
d = edit_distance_2(X, Y, len(X), len(Y))
print(f"Distancia obtenida: {d}. Esperada: 2")

X, Y = "ola", "hola"
d = edit_distance_2(X, Y, len(X), len(Y))
print(f"Distancia obtenida: {d}. Esperada: 1")

X, Y = "silla", "escritorio"
d = edit_distance_2(X, Y, len(X), len(Y), d = 3, a = 4)
print(f"Distancia obtenida: {d}. Esperada: 27")

Distancia obtenida: 2. Esperada: 2
Distancia obtenida: 1. Esperada: 1
Distancia obtenida: 27. Esperada: 27


## Ejercicio 3

Quita todas las llamadas recursivas y llena la tabla de manera iterativa (como `knapsack_memo_2`).

In [234]:

def edit_distance_3(X, Y, m, n, a=1, d=1, memo=None):
    # Creacion de una tabla para almacenar los resultados de los subproblemas.
    if memo is None:
        memo = [[0 for x in range(n + 1)] for x in range(m + 1)]
      
    for i in range(m + 1):
        for j in range(n + 1):
  
            #Si la primera cadena es vacia, la unica opcion
            #es insertar todos los caracteres de la segunda cadena        
            if i == 0:
                memo[i][j] = j * d    # Min. operations = j
              
            #Si la segunda cadena es vacia, la unica opcion
            #es eliminar todos los caracteres de la primera cadena
            elif j == 0:
                memo[i][j] = i * d
              
            # Si los ultimos caractares de las dos cadenas son los mismo.            
            elif X[i-1] == Y[j-1]:
                memo[i][j] = memo[i-1][j-1]
              
            # Si los ultimos caractares son diferentes, considerar
            # todas las posibilidades y encontrar el minimo.
            else:
                memo[i][j] = min(memo[i][j-1] + d,
                                   memo[i-1][j] + d,
                                   memo[i-1][j-1] + a)
  
    return memo[m][n]

Verifica tus resultados:

In [242]:
X, Y = "casa", "raza"
d = edit_distance_3(X, Y, len(X), len(Y))
print(f"Distancia obtenida: {d}. Esperada: 2")

X, Y = "ola", "hola"
d = edit_distance_3(X, Y, len(X), len(Y))
print(f"Distancia obtenida: {d}. Esperada: 1")

X, Y = "silla", "escritorio"
d = edit_distance_3(X, Y, len(X), len(Y), d = 3, a = 4)
print(f"Distancia obtenida: {d}. Esperada: 27")

Distancia obtenida: 2. Esperada: 2
Distancia obtenida: 1. Esperada: 1
Distancia obtenida: 27. Esperada: 27


## References 
- https://www.geeksforgeeks.org/edit-distance-dp-5/
- https://en.wikipedia.org/wiki/Edit_distance
- https://web.stanford.edu/class/cs124/lec/med.pdf
- https://colab.research.google.com/drive/14--aBFHwZv4QPNyAfYOM7HBRY56wkGGf?usp=sharing